In [1]:
import json
from json import JSONEncoder
import re
import time

In [10]:
class LogMetrics:
    def __init__(self):
        self.log_file_name=""
        self.dir={}
        self.metrics=[]
        self.error=False
        self.error_message=""

In [3]:
class Encoder(JSONEncoder):
    def default(slef,o):
        return o.__dict__

In [40]:
def process_log_file(file_list):
    
    Pattern_dir=r'(?im)^\s+(?P<type>source|Dest)\s+:\s+(?P<path>.+)'
    
    pattern_error=r'(?im)^(?P<Timestamp>\d{4}/\d{2}/\d{2})\s+(?P<error>.+)'
    
    pattern_metrics=r'(?im)(?P<type>Dirs|Files|Bytes)\s+:'\
                        '\s+(?P<total>\d+)\s+(?P<copied>\d+)'\
                        '\s+(?P<skipped>\d+)\s+(?P<mismatch>\d+)'\
                        '\s+(?P<failed>\d+)\s+(?P<extras>\d+)'
    
    for file in file_list:
            log_metrics=LogMetrics()
            log_metrics.file_name=file
            
            with open(file,'r',encoding='utf-8') as rdr:
                for line in rdr:
                    match=re.search(pattern_error,line)
                    
                    if match:
                        log_metrics.error=True
                        log_metrics.error_message=line
                    
                    match=re.search(Pattern_dir,line)
                    if match:
                        log_metrics.dir[match.group('type')]=match.group('path')
                    
                    match=re.search(pattern_metrics,line)
                    if match:
                        metricsRow=[]
                        metricsHeader=[]
                        captureHeader=False
                        
                        if len(log_metrics.metrics)==0:
                            captureHeader=True
                            
                        for key,value in match.groupdict().items():
                            if captureHeader:
                                metricsHeader.append(key)
                                    
                                metricsRow.append(value)
                                
                        if captureHeader:
                            log_metrics.metrics.append(metricsHeader)
                            
                        log_metrics.metrics.append(metricsRow)
                        
            with open(file+".json","w",encoding='utf-8') as wr:
                json.dump(log_metrics,wr,ensure_ascii=False,cls=Encoder,indent=True)
                        
    
    return

In [41]:
file_list=['Robocopy_success.txt','Robocopy_invalid.txt']
startime=time.time()

process_log_file(file_list)

endtime=time.time()

print("elapsed Time : {0:.6f}".format(endtime-startime))


['Dirs', '7', '6', '1', '0', '0', '0']
[]
[]
elapsed Time : 0.004995
